In [24]:
import pandas as pd
import numpy as np
import statsmodels.api as sm # para Regresión Lineal

# Cargar el dataset maestro generado en la Fase 1
df_master = pd.read_csv('../01_data/output/master_df.csv')
# Paso inicial: Nos enfocaremos en un solo país (ej. México) y un producto clave (ej. StockCode '22423')
# Esto simplifica el análisis visual.
df_mexico_product = df_master[
    (df_master['Country'] == 'MEXICO') & (df_master['StockCode'] == 22423)
]

In [25]:
# Carga la tabla maestra (usando la corrección de ruta de la última vez)
df_master = pd.read_csv('../01_data/output/master_df.csv')

# 1. Agrupa y suma la cantidad total vendida de cada producto
sales_by_stock = df_master.groupby('StockCode')['Total_Quantity_Sold'].sum()

# 2. Encuentra el producto más vendido y el segundo más vendido
top_stocks = sales_by_stock.nlargest(5)
print("Los 5 productos más vendidos globalmente son:")
print(top_stocks)

# 3. **¡Elegir un nuevo código!**
# Elige el primer o segundo código de la lista (ej. 85123A o 22423)
# Usaremos el primero de la lista para tener más datos.
NEW_STOCK_CODE = top_stocks.index[0] 
print(f"\nProducto seleccionado para el análisis: {NEW_STOCK_CODE}")

Los 5 productos más vendidos globalmente son:
StockCode
85099B    28097
84879     26682
85123A    22594
84077     22041
22197     20697
Name: Total_Quantity_Sold, dtype: int64

Producto seleccionado para el análisis: 85099B


In [26]:
import statsmodels.api as sm 
import numpy as np

# Definir el País Objetivo y el Producto Objetivo
TARGET_COUNTRY = 'MEXICO' # País simulado
# Usamos el código más vendido que se acaba de determinar
TARGET_STOCK = NEW_STOCK_CODE 

print(f"Iniciando análisis de elasticidad para el producto {TARGET_STOCK} en {TARGET_COUNTRY}...")

# 1. Filtrar los datos para el análisis
df_mexico_product = df_master[
    (df_master['Country'] == TARGET_COUNTRY) & 
    (df_master['StockCode'] == TARGET_STOCK)
]

# 2. Filtrar para la Regresión Log-Log (Demanda > 0 y Precio > 0)
df_model = df_mexico_product[
    (df_mexico_product['Total_Quantity_Sold'] > 0) & 
    (df_mexico_product['UnitPrice'] > 0)
].copy()

# 3. Verificación final de registros
if df_model.empty:
    print(f"❌ ERROR: A pesar de ser un Top Seller, el producto {TARGET_STOCK} no tiene ventas válidas en {TARGET_COUNTRY}.")
    print("Intenta cambiando el TARGET_COUNTRY a 'BRASIL' o 'Otro_LATAM' en la celda anterior.")
else:
    print(f"Registros válidos para modelado: {len(df_model)}")

    # 4. Transformación Logarítmica
    df_model['log_Quantity'] = np.log(df_model['Total_Quantity_Sold'])
    df_model['log_Price'] = np.log(df_model['UnitPrice'])

    # 5. Definir variables y entrenar el Modelo OLS (Regresión Log-Log)
    X = sm.add_constant(df_model['log_Price']) 
    Y = df_model['log_Quantity'] 

    model = sm.OLS(Y, X).fit()

    # 6. Obtener la Elasticidad (el coeficiente de log_Price)
    elasticity = model.params['log_Price']

    # 7. Presentación de Resultados
    print("\n" + "="*70)
    print(f"✅ ELASTICIDAD PRECIO para {TARGET_STOCK} en {TARGET_COUNTRY} ({df_model['Local_Currency'].iloc[0]})")
    print(f" Elasticidad (β₁): {elasticity:.4f}")
    print("="*70)
    
    if elasticity < -1:
        print("INTERPRETACIÓN: Producto **ELÁSTICO** (Muy sensible al precio). Bajar el precio ligeramente aumentará los ingresos por volumen.")
    elif elasticity > -1 and elasticity < 0:
        print("INTERPRETACIÓN: Producto **INELÁSTICO** (Poco sensible al precio). ¡Oportunidad de Margen! Sube el precio para maximizar las ganancias.")
    else:
        print("INTERPRETACIÓN: Elasticidad atípica o nula. Sugiere que otros factores (como la estacionalidad) dominan la demanda.")
    
    # Muestra el resumen estadístico completo (opcional, para el reporte técnico)
    # print(model.summary())

Iniciando análisis de elasticidad para el producto 85099B en MEXICO...
Registros válidos para modelado: 1285

✅ ELASTICIDAD PRECIO para 85099B en MEXICO (MXN)
 Elasticidad (β₁): -8.6859
INTERPRETACIÓN: Producto **ELÁSTICO** (Muy sensible al precio). Bajar el precio ligeramente aumentará los ingresos por volumen.


In [27]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# --- A. PREPARACIÓN DEL DATASET COMPLETO ---

# 1. Variables Categóricas
# Convertimos las variables categóricas (StockCode, Country, Month, DayOfWeek) a formato numérico (One-Hot Encoding)
# Esto es necesario para XGBoost.
df_model_full = pd.get_dummies(df_master.drop(columns=['InvoiceDate', 'Local_Currency']), 
                               columns=['StockCode', 'Country'])

# 2. Definir Variables (Features) y Target
# X = Variables predictoras: Precio propio, Precio del competidor y factores de tiempo/geografía
X = df_model_full.drop(columns=['Total_Quantity_Sold'])
# Y = Variable objetivo: La demanda que queremos predecir
Y = df_model_full['Total_Quantity_Sold']

# 3. División Entrenamiento/Prueba
# Separamos los datos para evaluar la precisión del modelo
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

# --- B. ENTRENAMIENTO DEL MODELO XGBOOST ---

# 4. Inicializar y Entrenar el Modelo
# XGBoost es un modelo de ensamble (potenciación del gradiente) muy preciso.
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, Y_train)

# 5. Evaluación
Y_pred = xgb_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))

print("="*60)
print(f"✅ Modelo de Predicción de Demanda (XGBoost) Entrenado")
print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse:.2f}")
print("Este RMSE representa el error promedio en unidades al predecir la demanda.")
print("="*60)

✅ Modelo de Predicción de Demanda (XGBoost) Entrenado
Raíz del Error Cuadrático Medio (RMSE): 13.28
Este RMSE representa el error promedio en unidades al predecir la demanda.


In [28]:
# --- C. FUNCIÓN DE OPTIMIZACIÓN DE PRECIOS (CORREGIDA) ---

# 1. Definir el Costo (Asumimos que el Costo Fijo es 50% del Precio Promedio)
average_price = df_master['UnitPrice'].mean()
FIXED_COST = average_price * 0.50 

def calculate_optimized_price(product_features):
    """
    Busca el precio óptimo para un producto específico probando un rango de precios.
    """
    best_profit = -np.inf
    optimal_price = 0

    # Probaremos precios en un rango del -20% al +20% del precio actual
    price_range = np.linspace(
        product_features['UnitPrice'].iloc[0] * 0.8, 
        product_features['UnitPrice'].iloc[0] * 1.2, 
        num=50 # Probamos 50 puntos de precio
    )

    for price in price_range:
        # Clonar las features para la predicción
        features_to_predict = product_features.copy()
        
        # Actualizar el precio propio en las features
        features_to_predict['UnitPrice'] = price
        
        # Predecir la Demanda con el nuevo precio
        # ¡CORRECCIÓN AQUÍ! -> Eliminamos .drop(columns=['Total_Quantity_Sold'])
        predicted_demand = xgb_model.predict(features_to_predict)[0] 
        
        # Calcular la Ganancia
        profit = (price - FIXED_COST) * predicted_demand
        
        # Si esta ganancia es mejor, actualizamos el precio óptimo
        if profit > best_profit:
            best_profit = profit
            optimal_price = price
            
    return optimal_price, best_profit